In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# The goal is not in classification really, because the metric is not accuracy, but AUC, so simple LinearRegression gives much better score than LogisticRegression, better scores i managed to get only from neural networks, but there are enough examples of them already

In [ ]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score

from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')

#https://www.kaggle.com/camnugent/geospatial-feature-engineering-and-visualization

In [ ]:
df1 = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv', index_col = 'id')

X_final1 = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv', index_col = 'id')

In [ ]:
X_test = df1.copy()
X_final = X_final1.copy()

In [ ]:
X_test = df1.copy()
X_test = df1.sample(frac=1)

In [ ]:
agg_features = ['sum','mean','std','max','min','kurt']
features = X_test.keys().tolist()[:-1]

for ft in agg_features:
    
    class_method = getattr(pd.DataFrame, ft)
    X_test[ft] = class_method(X_test[features], axis=1)
    X_final[ft] = class_method(X_final[features], axis=1)

In [ ]:
X = X_test.drop('target', axis=1)
y = X_test['target']

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)

cols = X.columns


X = scaler.transform(X)
X_final = scaler.transform(X_final)

X = pd.DataFrame(X, columns = cols)
X_final = pd.DataFrame(X_final, columns = cols)
X_final.index = X_final1.index

In [ ]:
%%time
#building a price prediction
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state = 1)

model = LogisticRegression(solver='liblinear', n_jobs = 4)

model.fit(X_train, y_train)
y_pred_test = model.predict(X_valid)
print(roc_auc_score(y_valid, y_pred_test))



In [ ]:
%%time
#building a price prediction
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression


model = LinearRegression()

model.fit(X_train, y_train)
y_pred_test = model.predict(X_valid)
print(roc_auc_score(y_valid, y_pred_test))

Creating submission file

In [ ]:
model = LinearRegression()

model.fit(X, y)
y_pred_test = model.predict(X_final)
y_pred_df = pd.DataFrame(y_pred_test, columns=['target'])
y_pred_df['id'] = X_final.index

In [ ]:
columns = ['id','target']

y_pred_df[columns]

In [ ]:
filename = 'target.csv'
y_pred_df[columns].to_csv(filename,index = None)